In [ ]:
from __future__ import absolute_import, division, print_function

import sys
import os
import time
from math import ceil, floor, sqrt

sys.path.insert(0,'../../')
sys.path.insert(0,'../../../py')

import parameters
import utilities
import output_utilities
from spectrogram_utilities import spectrogram_plot, spectrogram_reshape
from spectrogram_output_visualiser import spectrogram_and_output, spectrogram_cut
import spectrogram_cuts_db_creation


import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt

import tensorflow as tf
import keras

# matplotlib.get_backend()
%matplotlib inline

# constants
awd_event = 1
site = parameters.sites[0]

# use gpu
from keras import backend as K
print('GPUs:',K.tensorflow_backend._get_available_gpus())
gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
print('FUCK')

In [1]:
# get file
site = parameters.sites[0]
file = utilities.random_file(awd_event, site)
# file = '2013-08-14UT21:11:41.44611906.marion.vr2'
file = '2013-07-25UT05:24:35.84611922.marion.vr2'
print(file)
_, time, freq, spec = spectrogram_and_output(awd_event, site, file, output=True, zscore=False)

_, time, freq, spec = spectrogram_and_output(awd_event, site, file, output=False, zscore=True)
_ = spectrogram_plot(spec,x_axis=time, y_axis=freq/1000, size=(16,5))

heigth = 2
width = 3*heigth
_spec = spec.copy()
f_max, t_max = spec.shape
t_index = np.array(list(range(0, t_max, ceil(t_max/width))))
f_index = np.array(list(range(0, f_max, ceil(f_max/heigth))))
for t in t_index:
#     print(t)
    _spec[:,t] = np.full(spec[:,0].shape, spec.min())
for f in f_index:
    _spec[f,:] = np.full(spec[0,:].shape, spec.min())
#         print((t,f), (t+int(t_max/S),f+int(f_max/S)))
_ = spectrogram_plot(_spec, size=(16,5))
# print((t_max,f_max))
# print(t_index)
# print(f_index)

# load neural network
model = keras.models.load_model('cnn_culmul.h5')

cuts = []
pred_cuts = []
result = []
for t in range(len(t_index)):
    for f in range(len(f_index)):
        # get cut coordinate
        cut = []
        if t < len(t_index)-1 and f<len(f_index)-1:
            cut = [f_index[f],f_index[f+1],t_index[t],t_index[t+1]]
        elif f == len(f_index)-1 and t < len(t_index)-1:
            cut = [f_index[f],f_max,t_index[t],t_index[t+1]]
        elif t==len(t_index)-1 and f < len(f_index)-1:
            cut = [f_index[f],f_index[f+1],t_index[t],t_max]
        else:
            cut = [f_index[f],f_max,t_index[t],t_max]

        spec_cut = _spec[cut[0]:cut[1], cut[2]:cut[3]]
        spec_cut = spectrogram_reshape(spec_cut).flatten().reshape(1,parameters.clf_input_width*parameters.clf_input_height)
#         spec_cut = scaler.transform(spec_cut)
        spec_cut = spec_cut.reshape(parameters.clf_input_depth,
              parameters.clf_input_width,
              parameters.clf_input_height)
        pred = model.predict(spec_cut)
        pred = np.argmax(pred, axis=1)

        
        cuts.append(cut)
        pred_cuts.append(pred)
        
        result.append([cut,pred])
        
pred_cuts = np.array(pred_cuts).reshape(heigth,width)
result = np.array(result)
print(result)

NameError: name 'parameters' is not defined